# Ensemble Learning and Random Forests

Random Forest: A bunch of decision trees trained on random subsets of the data 
    - Tends to beat single trees trained on full dataset

A good idea is to train a few different good models and then combine them making one ensemble model

Can weight each model's contribution to the ensemble based on its test accuracy

## Voting Classifiers

Example: you have trained a logistic regression, svm classifier, and random forest. Each are around 80% accuracy

Hard Voting Classifier: each model counts as a vote, the ensemble prediction is whichever class gets the most votes (simple majority)

Voting works well if you have models that are "good" at different classes (diverse models)

Get more diverse models by training them differently. Play with hyperparams or train on different subsets of data.

In [ ]:
from sklearn.datasets import make_moons
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

X, y = make_moons(500, noise=.3, random_state=42)
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, random_state=42)

votingClf = VotingClassifier(
    estimators=[
        ("log", LogisticRegression(random_state=42)),
        ("rf", RandomForestClassifier(random_state=42)),
        ("svc", SVC(random_state=42))
    ]
)

votingClf.fit(Xtrain, ytrain)
votingClf.p

,"estimators estimators: list of (str, estimator) tuplesInvoking the ``fit`` method on the ``VotingClassifier`` will fit clonesof those original estimators that will be stored in the class attribute``self.estimators_``. An estimator can be set to ``'drop'`` using:meth:`set_params`... versionchanged:: 0.21 ``'drop'`` is accepted. Using None was deprecated in 0.22 and support was removed in 0.24.","[('log', ...), ('rf', ...), ...]"
,"voting voting: {'hard', 'soft'}, default='hard'If 'hard', uses predicted class labels for majority rule voting.Else if 'soft', predicts the class label based on the argmax ofthe sums of the predicted probabilities, which is recommended foran ensemble of well-calibrated classifiers.",'hard'
,"weights weights: array-like of shape (n_classifiers,), default=NoneSequence of weights (`float` or `int`) to weight the occurrences ofpredicted class labels (`hard` voting) or class probabilitiesbefore averaging (`soft` voting). Uses uniform weights if `None`.",None
,"n_jobs n_jobs: int, default=NoneThe number of jobs to run in parallel for ``fit``.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionadded:: 0.18",None
,"flatten_transform flatten_transform: bool, default=TrueAffects shape of transform output only when voting='soft'If voting='soft' and flatten_transform=True, transform method returnsmatrix with shape (n_samples, n_classifiers * n_classes). Ifflatten_transform=False, it returns(n_classifiers, n_samples, n_classes).",True
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting will be printed as itis completed... versionadded:: 0.23",False
,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001


In [2]:
for name, estimator in votingClf.named_estimators_.items():
    print(name, estimator.score(Xtest, ytest))

log 0.864
rf 0.896
svc 0.896


In [3]:
votingClf.score(Xtest, ytest)
# Hard voting gets about 1.5% better than individual models

0.912

#### Soft Voting

If the individual models have a .predict_proba() method, you can soft vote, which uses the averaged predicted probas to decide the correct class. This is usually better than hard voting.

In [4]:
votingClf.voting = "soft"
votingClf.named_estimators['svc'].probability = True
votingClf.fit(Xtrain, ytrain)
votingClf.score(Xtest, ytest)

# if predicted probs are not well calibrated, you can use sklearn.calibration.CalibratedClassifierCV to calibrate

0.92

## Bagging and Pasting
Idea: use multiple instances of the same training algorithm, but different subsets of the data

Bagging: Sampling data w/ replacement (different models can have very similar data subsets)

Pasting: sampling data without replacement (every data subset is fully unique)

For classification, take the mode prediction. For regression, take the average prediction.

In practice, this ensemble ends up with similar bias but lower variance than single models. So its a good idea to pick models with high variance/low bias to use with this model (trees)

Bagging > pasting when data is noisy or model overfits easily (deep trees). Otherwise, pasting > bagging.

In [5]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bagClf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500, max_samples=100, n_jobs=-1, random_state=42)
bagClf.fit(Xtrain,ytrain)
# bagging classifiers automatically do soft voting

,"estimator estimator: object, default=NoneThe base estimator to fit on random subsets of the dataset.If None, then the base estimator is a:class:`~sklearn.tree.DecisionTreeClassifier`... versionadded:: 1.2 `base_estimator` was renamed to `estimator`.",DecisionTreeClassifier()
,"n_estimators n_estimators: int, default=10The number of base estimators in the ensemble.",500
,"max_samples max_samples: int or float, default=NoneThe number of samples to draw from X to train each base estimator (withreplacement by default, see `bootstrap` for more details).- If None, then draw `X.shape[0]` samples irrespective of `sample_weight`.- If int, then draw `max_samples` samples.- If float, then draw `max_samples * X.shape[0]` unweighted samples or `max_samples * sample_weight.sum()` weighted samples.",100
,"max_features max_features: int or float, default=1.0The number of features to draw from X to train each base estimator (without replacement by default, see `bootstrap_features` for moredetails).- If int, then draw `max_features` features.- If float, then draw `max(1, int(max_features * n_features_in_))` features.",1.0
,"bootstrap bootstrap: bool, default=TrueWhether samples are drawn with replacement. If False, sampling withoutreplacement is performed. If fitting with `sample_weight`, it isstrongly recommended to choose True, as only drawing with replacementwill ensure the expected frequency semantics of `sample_weight`.",True
,"bootstrap_features bootstrap_features: bool, default=FalseWhether features are drawn with replacement.",False
,"oob_score oob_score: bool, default=FalseWhether to use out-of-bag samples to estimatethe generalization error. Only available if bootstrap=True.",False
,"warm_start warm_start: bool, default=FalseWhen set to True, reuse the solution of the previous call to fitand add more estimators to the ensemble, otherwise, just fita whole new ensemble. See :term:`the Glossary `... versionadded:: 0.17 *warm_start* constructor parameter.",False
,"n_jobs n_jobs: int, default=NoneThe number of jobs to run in parallel for both :meth:`fit` and:meth:`predict`. ``None`` means 1 unless in a:obj:`joblib.parallel_backend` context. ``-1`` means using allprocessors. See :term:`Glossary ` for more details.",-1
,"random_state random_state: int, RandomState instance or None, default=NoneControls the random resampling of the original dataset(sample wise and feature wise).If the base estimator accepts a `random_state` attribute, a differentseed is generated for each instance in the ensemble.Pass an int for reproducible output across multiple function calls.See :term:`Glossary `.",42
,"verbose verbose: int, default=0Controls the verbosity when fitting and predicting.",0


##### Out of Bag Evaluation

Each individual predictor only sees about 63% of the data, the remaining 37% are called out-of-bag instances. Every predictor has different oob instances.

You can evaluate a model with OOB score built-in:

In [6]:
bagClf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500, max_samples=100, n_jobs=-1, random_state=42, oob_score=True)
bagClf.fit(Xtrain,ytrain)
bagClf.oob_score_

0.9253333333333333

In [7]:
from sklearn.metrics import accuracy_score

accuracy_score(bagClf.predict(Xtest), ytest)

# The book showed OOB < true accuracy, but this test shows OOB was above true accuracy. Interesting that the bagClf's true performance is a little poor here.

0.904

In [8]:
bagClf.oob_decision_function_[:3] # predicted probs

array([[0.35579515, 0.64420485],
       [0.43513514, 0.56486486],
       [1.        , 0.        ]])

##### Random Patches and Random Subspaces
You can randomly sample features as well as instances. `max_features` and `bootstrap_features` work similarly to `max_samples` and `bootstrap`

This works well for high dimension data (images) to speed up training. 

A random patch is a subset of instances and features (for an n x m input, a x b random patch where a<n and b<m)

A random subspace is keeping all instances but sampling the features. set bootstrap=False and max_samples=1.0, and set bootstrap_features and max_features

This increases predictor diversity, lowering variance. As always, you are trading less variance for more bias.

## Random Forests

A random forest is an ensemble of decision trees, usually trained via bagging. 

RandomForestClassifier = DecisionTreeClassifier --> BaggingClassifier 

In [9]:
from sklearn.ensemble import RandomForestClassifier

# Random Forest Classifier has all the arguments of DecisionTree and BaggingClassifier, so control regularization with max_depth, max_leaf_nodes, etc
rfClf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1, random_state=42)
rfClf.fit(Xtrain, ytrain)
preds = rfClf.predict(Xtest)

accuracy_score(preds, ytest)

0.912

Random Forest works a little differently than a regular decision tree. Instead of looking for the best split, it looks for the best split among a subset of features, in order to increase tree diversity. 

##### Extra-Trees

For each tree in a random forest, a random subset of features is considered when splitting. You can make trees even MORE random by setting `splitter=random` in DecisionTreeClassifier. This sets random thresholds for each feature rather than searching for the best ones.

Again, this trades lower variance for higher bias. Works well with noisy data or situations where RandomForest is overfitting.

In [10]:
extremelyRandomized = BaggingClassifier(
    DecisionTreeClassifier(max_features="sqrt", splitter="random", max_leaf_nodes=16),
    n_estimators=500,
    n_jobs=-1,
    random_state=42,
    bootstrap=False
)

# there is also an sklearn ExtraTreesClassifier

##### Feature Importance

Sklearn measures feature importance by looking at all the tree nodes that use that feature, and then averaging how much those nodes reduce impurity. Its a weighted avg, so more samples in node = feature is better.

Sklearn computes these automatically:

In [11]:
from sklearn.datasets import load_iris
iris = load_iris(as_frame=True)
rfClf = RandomForestClassifier(n_estimators=1000)
rfClf.fit(iris.data, iris.target)

for score, name in zip(rfClf.feature_importances_, iris.data.columns):
    print(f"{name} : {round(score,2)}")

# this makes feature selection very easy! it looks like sepal width is the worst predictor by far

ImportError: load_iris with as_frame=True requires pandas.

## Boosting

Boosting: any ensemble method that combines weak learners into a strong learner

Train predictors sequentially, where the next predictor learns where the previous one struggled

##### AdaBoost (adaptive boosting)

Idea: where the last model underfit, this model will focus. The newest predictors focus more and more on the hardest cases / decision boundaries.

Train a classifier > predict on training set > increase weight of training instances that were predicted wrong > go back to step 1

Important con: sequential instead of parallel training is slow.

Predictors are weighted based on their training set accuracy.

Weighted error rate of the j'th predictor:
$$r_j = {\sum_{i=1}^{m} w^{(j)}_i \cdot \mathbb{1}(y_i \neq \hat{y}_i^{(j)})}$$
${\hat{y}_j}^{(i)}$ is the j'th predictors prediction for the i'th instance

The weight is then calculated via: (alpha j is weight of predictor j, eta is the learning rate)
$$\alpha_j = \eta \log\left(\frac{1-r_j}{r_j}\right)$$

Weight update rute:
- if the prediction is right, no weight change
- if the prediction is wrong, ${w^{(i)}} = {w^{(i)}}{e^{(\alpha_j)}}$

Lastly, all weights get normalized so they sum to 1. Predictions are made by weighting the predictors by $\alpha_j$

$$\hat{y}(\mathbf{x}) = \underset{k}{argmax} \sum_{j=1}^{N} \alpha_j \cdot \mathbb{1}(\hat{y}_j(\mathbf{x}) = k)$$

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

adaClf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), # this is the default, a "decision stump"
    n_estimators=100,
    learning_rate=0.5,
    random_state=42,
    algorithm="SAMME" # this is the default, and is modified from the adaboost given above
)

adaClf.fit(Xtrain, ytrain)
adaClf.score(Xtest, ytest)

0.896

## Gradient Boosting

Similar conceptually to AdaBoost, but instead of tweaking weights, Gradient Boosting fits the new predictor to the *residual errors* made by the previous predictor.

In [ ]:
# Gradient Boosting walkthru

import numpy as np
from sklearn.tree import DecisionTreeRegressor

m = 100 # n instances
rng = np.random.default_rng(seed=42)
X = rng.random((m,1)) - 0.5
noise = .05 * rng.standard_normal(m)
y = 3 * X[:,0]**2 + noise

treeReg1 = DecisionTreeRegressor(max_depth=2, random_state=42)
treeReg1.fit(X,y)

y2 = y - treeReg1.predict(X)
treeReg2 = DecisionTreeRegressor(max_depth=2, random_state=42)
treeReg2.fit(X,y2)

y3 = y2 - treeReg2.predict(X)
treeReg3 = DecisionTreeRegressor(max_depth=2, random_state=42)
treeReg3.fit(X,y2)

# 1 predicts the val, 2 predicts residuals, 3 predicts residuals of residuals, and so on

DecisionTreeRegressor(max_depth=2, random_state=42)

In [ ]:
# Making predictions
Xnew = np.array([[-.4], [0], [.5]])
sum(tree.predict(Xnew) for tree in [treeReg1, treeReg2, treeReg3])

array([0.54781022, 0.06427754, 0.98721801])

In [ ]:
# Equivalent to above
from sklearn.ensemble import GradientBoostingRegressor

gradReg = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1, random_state=42)
gradReg.fit(X,y)
gradReg.predict(Xnew)
# performs just as poorly (this is a small n of estimators and depth)

array([0.57356534, 0.0405142 , 0.66914249])

`learning_rate` scales how much each tree contributes. Low eta = more trees needed, but generalizes better. This is called *shrinkage* and is regularization. Use cross-val to find best learning rate.

To find the optimal number of trees, set `n_iter_no_change` hyperparam. If you get no changes in 10 new trees, gradient boosting will stop.

In [ ]:
gradStop = GradientBoostingRegressor(max_depth=2, learning_rate=.05, n_estimators=500, n_iter_no_change=10, random_state=42)
gradStop.fit(X,y)
gradStop.n_estimators_
# when n_iter_no_change is set, sklearn makes a small validation set with the training data to figure out if theres change. tol= hyperparam controls how much you need to see to count as change

53

In [ ]:
# Stochastic Gradient Boosting
GradientBoostingRegressor(max_depth=2, learning_rate=.05, n_estimators=500, subsample=0.25, random_state=42)

#subsample trains each successive tree on that fraction of the data. High bias, lower variance, faster training.

GradientBoostingRegressor(learning_rate=0.05, max_depth=2, n_estimators=500,
                          random_state=42, subsample=0.25)

#### Histogram-based Gradient Boosting

HGB is good for huge datasets. It bins the input features and replaces them with integers. N bins is controlled by max_bins= hyperparam, default is 255 (and it cant be higher than 255). Makes checking thresholds much faster.

HGB is O(b x m), grad boost is usually O(n x m x log(m)). b=bins, n=features, m=instances.


In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
# Early stopping default turned on for m > 10000 
# No subsampling
# n_estimators becomes max_iter
# Minimal number of hyperparams to tweak

# Supports missing vals and categorical vals by default

In [ ]:
# pipeline example for HGB -> based on california dataset ch2
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.preprocessing import OrdinalEncoder

hgbReg = make_pipeline(
    make_column_transformer((OrdinalEncoder(), ["ocean_proximity"]),
                            remainder="passthrough"),
    HistGradientBoostingRegressor(categorical_features=[0], random_state=42)
)

### Other Gradient Boosting

XGBoost, CatBoost, and LightGBM are all gradient boosting libraries that perform very well.

## Stacking (stacked generalization)

What if instead of some simple voting algorithm, we trained a model to do the aggregation?

To train the "blender" (algo that replaces voting):
 - Use cross_val_predict() on every predictor in ensemble to get out of sample training set predictions
 - Feed these predictions into the blender -> try to predict OG target vals

You could also train multiple blenders on top of all the original predictions, and then a blender for the blenders. Keep in mind the accuracy gains for doing this are likely very small.

In [ ]:
from sklearn.ensemble import StackingClassifier

stackingClf = StackingClassifier(
    estimators = [
        ("lr", LogisticRegression()),
        ("rf", RandomForestClassifier()),
        ("svc", SVC(probability=True))
    ],
    final_estimator=RandomForestClassifier(),
    cv=5
)
stackingClf.fit(Xtrain, ytrain)
stackingClf.score(Xtest, ytest)
# Not a bad final score !

0.904

| Ensemble method | When to use it | Example use cases |
|---|---|---|
| Hard voting | Balanced classification dataset with multiple strong but diverse classifiers. | Spam detection, sentiment analysis, disease classification |
| Soft voting | Classification dataset with probabilistic models, where confidence scores matter. | Medical diagnosis, credit risk analysis, fake news detection |
| Bagging | Structured or semi-structured dataset with high variance and overfitting-prone models. | Financial risk modeling, ecommerce recommendation |
| Pasting | Structured or semi-structured dataset where more independent models are needed. | Customer segmentation, protein classification |
| Random forest | High-dimensional structured datasets with potentially noisy features. | Customer churn prediction, genetic data analysis, fraud detection |
| Extra-trees | Large structured datasets with many features, where speed is critical and reducing variance is important. | Real-time fraud detection, sensor data analysis |
| AdaBoost | Small to medium-sized, low-noise, structured datasets with weak learners (e.g., decision stumps), where interpretability is helpful. | Credit scoring, anomaly detection, predictive maintenance |
| Gradient boosting | Medium to large structured datasets where high predictive power is required, even at the cost of extra tuning. | Housing price prediction, risk assessment, demand forecasting |
| Histogram-based gradient boosting (HGB) | Large structured datasets where training speed and scalability are key. | Click-through rate prediction, ranking algorithms, real-time bidding in advertising |
| Stacking | Complex, high-dimensional datasets where combining multiple diverse models can maximize accuracy. | Recommendation engines, autonomous vehicle decision-making, Kaggle competitions |